## Imports

In [1]:
import pandas as pd
import numpy as np
import json
import requests
from datetime import datetime

## Basic Simulated API Response Parsing

In [2]:
#Example 1- Parsing through dict (object)
plaid_response = {
    "income_summary": {
        "monthly_income": 5000,
        "income_sources": ["payroll", "freelance"]
    }
}

In [3]:
income = plaid_response["income_summary"]["monthly_income"]
income

5000

In [678]:
#Example 2- Parsing through list
credit_report = {
    "accounts": [
        {"type": "credit card", "status": "open", "date_opened": "2023-05-01"},
        {"type": "auto loan", "status": "closed", "date_opened": "2020-08-10"}
    ]
}

In [680]:
accounts = credit_report["accounts"]

open_accounts = []

for obj in accounts:
    if obj["status"] == "open":
        open_accounts.append(obj)
    
open_accounts

[{'type': 'credit card', 'status': 'open', 'date_opened': '2023-05-01'}]

## Sample Question 1

Prompt:
You're given a response from SEON's Fraud API. Implement a function that knocks out any applicant if:

Their fraud_score is above 80

They are listed on more than 2 watchlists

In [11]:
kyc_response = {
    "fraud_score": 75,
    "watchlists": ["OFAC", "Interpol", "EU Sanctions"]
}

In [13]:
#Input: kyc_response
#Output: Boolean T/F (True = knocked out(
def is_knockout_kyc(kyc_response):

    # IS fraud_score > 80? 
    fraud_score = kyc_response["fraud_score"]
    if (fraud_score > 80):
        return True;

    watchlist_array =  kyc_response["watchlists"]
    if (len(watchlist_array) > 2):
        return True;

    

In [15]:
is_knockout_kyc(kyc_response)

True

## Sample Question 2

Prompt:
Given a simplified Experian report, knock out applicants if:

Credit score < 600

All accounts are younger than 12 months

Use today’s date in the code.

In [17]:
credit_report = {
    "credit_score": 680,
    "accounts": [
        {"date_opened": "2024-11-01"},
        {"date_opened": "2025-02-15"}
    ]
}

In [302]:
#Doing it not in a function first.
accounts = credit_report["accounts"]

format_string = "%Y-%m-%d" #for converting datetime object

list_of_date_objects = [] 

for acc in accounts:
    
    #convert acc["date_opened"] to datetim.date object
    datetime_object = datetime.strptime(acc["date_opened"], format_string).date() 

    list_of_date_objects.append(datetime_object)

min_date = min(list_of_date_objects) #min (oldest) date 
today = date.today() # todays date 
min_date

months_diff = (today.year - min_date.year) * 12 + (today.month - min_date.month)
months_diff

8

In [304]:
#Input: JSON report
#Output: Boolean T/F, False = denied 
def is_credit_history_eligible(credit_report):

    #Check if credit_score < 600
    credit_score = credit_report["credit_score"]

    if(credit_score < 600):
        return False

    #Determine if all accounts are younger than 12months from today 
    accounts = credit_report["accounts"]

    format_string = "%Y-%m-%d" #for converting datetime object

    list_of_date_objects = [] 

    for acc in accounts:
    
        #convert acc["date_opened"] to datetim.date object
        datetime_object = datetime.strptime(acc["date_opened"], format_string).date() 

        list_of_date_objects.append(datetime_object)

    min_date = min(list_of_date_objects) #min (oldest) date 
    today = date.today() # todays date 

    months_diff = (today.year - min_date.year) * 12 + (today.month - min_date.month)

    if months_diff < 12:
        return False #too young of accounts

    return True 

In [306]:
is_credit_history_eligible(credit_report)

False

## Public API Example #1

In [25]:
your_name = input("Please enter your name: ") #ask user for input
print(f"Hello, {your_name}") #variable in a string

Please enter your name:  mike


Hello, mike


In [27]:
headers = {
    'accept': 'application/json'
}

params = { #query parameters (after the ?) 
    'name': your_name 
}

response = requests.get(
    #'https://api.genderize.io/?name=lauren'
    'https://api.genderize.io/',
    params = params,
    headers = headers
)

print(response.json())

{'count': 1145946, 'name': 'mike', 'gender': 'male', 'probability': 1.0}


In [29]:
json_response = response.json()
name = json_response["name"]
gender = json_response["gender"]

print(f"Hello, my name is {name} and I am {gender}.")

Hello, my name is mike and I am male.


## Public API Example #2

#### API : https://dogapi.dog/docs/api-v2

#### Q: Return all of the Names associated with each group

In [55]:
##API Call - get JSON Response

In [57]:
headers = {
    'accept': 'application/json'
}

#params = { #query parameters 
#    'name': your_name 
#}

response = requests.get(
    #'https://api.genderize.io/?name=lauren'
    'https://dogapi.dog/api/v2/groups',
    #params = params,
    headers = headers
)

In [59]:
json_response = response.json()

In [72]:
data_r = json_response["data"]

In [63]:
first_data_r = data_r[0]
first_data_r

{'id': '8000793f-a1ae-4ec4-8d55-ef83f1f644e5',
 'type': 'group',
 'attributes': {'name': 'Foundation Stock Service'},
 'relationships': {'breeds': {'data': [{'id': '68f47c5a-5115-47cd-9849-e45d3c378f12',
     'type': 'breed'},
    {'id': '30f62219-e225-42cd-bd07-02425f944c07', 'type': 'breed'},
    {'id': '087979f3-1c45-4d8a-a153-462bf5ea379e', 'type': 'breed'},
    {'id': 'dbff689b-8370-4b6a-9306-215aba549102', 'type': 'breed'},
    {'id': '399c6552-7b4f-418c-a3cc-951cf6752a2e', 'type': 'breed'},
    {'id': '112c4528-17e4-4e35-bcc3-706340d26d04', 'type': 'breed'},
    {'id': '4348208f-eb27-480f-bf3a-3f7c6b5de312', 'type': 'breed'},
    {'id': '8a78ce08-b3ae-4db0-9048-1354cddb7450', 'type': 'breed'},
    {'id': 'caf69af9-6fcb-4fb2-a543-d930513ac22a', 'type': 'breed'},
    {'id': 'c500abe0-0ee1-4209-9767-08cc03e6abdf', 'type': 'breed'},
    {'id': '1d18a7b3-ce08-41bb-a573-a99f89fb14f1', 'type': 'breed'},
    {'id': '44d9d8b1-184c-4ba8-84f1-c2cf582f9a95', 'type': 'breed'},
    {'id': 'c5

In [47]:
attributes = first_data_r["attributes"]
attributes

{'name': 'Foundation Stock Service'}

In [49]:
attributes["name"]

'Foundation Stock Service'

In [66]:
#Input JSON payload
#Output is a list of names  
def get_all_names(json_report):

    list_of_names = []
    
    #Get just "data" attributes
    json_data = json_report["data"]

    for obj in json_data:
        attributes = obj["attributes"]
        
        obj_name = attributes["name"]

        list_of_names.append(obj_name)

    return list_of_names

In [68]:
get_all_names(json_response)

['Foundation Stock Service',
 'Herding Group',
 'Hound Group',
 'Non-Sporting Group',
 'Sporting Group',
 'Terrier Group',
 'Toy Group',
 'Working Group',
 'Miscellaneous Class']

## Public API Example #3

#### Q: Given ID of a group, get the name and life expectancy of every breed in the group

GROUP ID = 8000793f-a1ae-4ec4-8d55-ef83f1f644e5

#### 1) API Request /groups{id} and get JSON

In [85]:
headers = {
    'accept': 'application/json'
}

#params = { #query parameters - after the question mark
#    'id': id 
#}

response = requests.get(
    'https://dogapi.dog/api/v2/groups/8000793f-a1ae-4ec4-8d55-ef83f1f644e5',
    #params = params,
    headers = headers
)

json_response_group = response.json()

#### 2) API Request /breeds{id} and get JSON
Will need to do this request over all breed IDs from group, but lets take one to start Breed ID = 30f62219-e225-42cd-bd07-02425f944c07

In [93]:
headers = {
    'accept': 'application/json'
}

#params = { #query parameters - after the question mark
#    'id': id 
#}

response = requests.get(
    'https://dogapi.dog/api/v2/breeds/30f62219-e225-42cd-bd07-02425f944c07',
    #params = params,
    headers = headers
)

json_response_breed = response.json()

In [95]:
json_response_breed

{'data': {'id': '30f62219-e225-42cd-bd07-02425f944c07',
  'type': 'breed',
  'attributes': {'name': 'Hokkaido',
   'description': 'The Hokkaido is a medium-sized breed of dog from the island of Hokkaido in Japan. These dogs are medium in size, with a thick double coat of fur to protect them from the cold. They have a dignified and loyal demeanor, making them excellent guard and companion dogs. They are intelligent and trainable, but require firm and consistent training to be obedient. They are brave and alert, with an instinct to protect their family and property.',
   'life': {'max': 15, 'min': 12},
   'male_weight': {'max': 30, 'min': 20},
   'female_weight': {'max': 30, 'min': 20},
   'hypoallergenic': False},
  'relationships': {'group': {'data': {'id': '8000793f-a1ae-4ec4-8d55-ef83f1f644e5',
     'type': 'group'}}}},
 'links': {'self': 'https://dogapi.dog/api/v2/breeds/30f62219-e225-42cd-bd07-02425f944c07'}}

#### 3) From this Breed JSON response, Get name and life expectancy of the Breed [Manual Try First]

In [119]:
json_breed_data_atts = json_response_breed["data"]["attributes"]

json_breed_data_atts

{'name': 'Hokkaido',
 'description': 'The Hokkaido is a medium-sized breed of dog from the island of Hokkaido in Japan. These dogs are medium in size, with a thick double coat of fur to protect them from the cold. They have a dignified and loyal demeanor, making them excellent guard and companion dogs. They are intelligent and trainable, but require firm and consistent training to be obedient. They are brave and alert, with an instinct to protect their family and property.',
 'life': {'max': 15, 'min': 12},
 'male_weight': {'max': 30, 'min': 20},
 'female_weight': {'max': 30, 'min': 20},
 'hypoallergenic': False}

In [121]:
breed_name = json_breed_data_atts["name"]
breed_lifeExp = json_breed_data_atts["life"]["max"]
breed_lifeExp 

15

#### 4) Function to Return Name and LifeExp of Breed Given JSON Response as Input

In [133]:
#Input: /breed{id} JSON Response
#Output: Object (Dict) of {breed_id:"",name:"",lifeExp""}
def breed_name_lifeExp(breed_json_response):

    breeds_name_lifeExp = {"breed_id": "",
                           "breed_name": "",
                           "breed_lifeExp": ""} #define output dictionary

    #Get the breed_id, breed_name and breed_lifeExp
    breed_id = breed_json_response["data"]["id"]
    
    breed_json_atts = breed_json_response["data"]["attributes"]

    breed_name = breed_json_atts["name"]
    breed_lifeExp = breed_json_atts["life"]["max"]

    #Add values to the corresponding key in Dictionary 
    breeds_name_lifeExp["breed_id"] = breed_id
    breeds_name_lifeExp["breed_name"] = breed_name
    breeds_name_lifeExp["breed_lifeExp"] = breed_lifeExp

    return breeds_name_lifeExp

In [135]:
#Test the function
example_breed = breed_name_lifeExp(json_response_breed)
example_breed

{'breed_id': '30f62219-e225-42cd-bd07-02425f944c07',
 'breed_name': 'Hokkaido',
 'breed_lifeExp': 15}

#### 5) Extrapolate to Loop Over All Breed IDs that you get from the Group:

##### 5a) Get List of all Breed IDs

In [183]:
parsed_json_group = json_response_group["data"]["relationships"]["breeds"]["data"]

breed_ids = []

for obj in parsed_json_group:

    breed_ids.append(obj["id"])

breed_ids = breed_ids[:5] #limit to 5
breed_ids

['68f47c5a-5115-47cd-9849-e45d3c378f12',
 '30f62219-e225-42cd-bd07-02425f944c07',
 '087979f3-1c45-4d8a-a153-462bf5ea379e',
 'dbff689b-8370-4b6a-9306-215aba549102',
 '399c6552-7b4f-418c-a3cc-951cf6752a2e']

##### 5b) Loop Over Breed ID List and put into Results

In [190]:
results = [] #output

for breed_id in breed_ids:
    
    #First, get the JSON Response
    headers = {
    'accept': 'application/json'
    }

    #params = { #query parameters - after the question mark
    #    'id': id 
    #}

    response = requests.get(
        'https://dogapi.dog/api/v2/breeds/'+breed_id,
        #params = params,
        headers = headers
    )

    json_response_breed = response.json()

    #Then, From json_response_breed, get the associated id, name, lifeExp
    one_breed_name_lifeExp = breed_name_lifeExp(json_response_breed)

    results.append(one_breed_name_lifeExp)

In [191]:
results

[{'breed_id': '68f47c5a-5115-47cd-9849-e45d3c378f12',
  'breed_name': 'Caucasian Shepherd Dog',
  'breed_lifeExp': 20},
 {'breed_id': '30f62219-e225-42cd-bd07-02425f944c07',
  'breed_name': 'Hokkaido',
  'breed_lifeExp': 15},
 {'breed_id': '087979f3-1c45-4d8a-a153-462bf5ea379e',
  'breed_name': 'Japanese Terrier',
  'breed_lifeExp': 20},
 {'breed_id': 'dbff689b-8370-4b6a-9306-215aba549102',
  'breed_name': 'Hanoverian Scenthound',
  'breed_lifeExp': 14},
 {'breed_id': '399c6552-7b4f-418c-a3cc-951cf6752a2e',
  'breed_name': 'Perro de Presa Canario',
  'breed_lifeExp': 12}]

#### 6) Find Breed With Highest Life Expectancy

In [200]:
lifeExp = 0

for obj in results:

    if lifeExp < obj["breed_lifeExp"]:
        lifeExp = obj["breed_lifeExp"]
    

In [202]:
lifeExp

20

In [694]:
longest_life_exp = 0
associated_breed_name = ""

for obj in results:
    
    if obj["breed_lifeExp"] > longest_life_exp:
        longest_life_exp = obj["breed_lifeExp"]
        associated_breed_name = obj["breed_name"]


In [696]:
associated_breed_name

'Caucasian Shepherd Dog'

## Public API Example #4
#### Q: Find the breed of dog that has the biggest difference between the max male weight and max female weight. Then output a list of the top 5.

#### 1) Get JSON Response /breeds

In [208]:
#Get JSON Response /breeds
headers = {
    'accept': 'application/json'
}

#params = { #query parameters 
#    'name': your_name 
#}

response = requests.get(
    'https://dogapi.dog/api/v2/breeds',
    #params = params,
    headers = headers
)

json_response_breed = response.json()

#### 2) Find Differntial Weight for 1 Breed

In [222]:
first_out = json_response_breed["data"][0]
first_out

{'id': '036feed0-da8a-42c9-ab9a-57449b530b13',
 'type': 'breed',
 'attributes': {'name': 'Affenpinscher',
  'description': 'The Affenpinscher is a small and playful breed of dog that was originally bred in Germany for hunting small game. They are intelligent, energetic, and affectionate, and make excellent companion dogs.',
  'life': {'max': 16, 'min': 14},
  'male_weight': {'max': 5, 'min': 3},
  'female_weight': {'max': 5, 'min': 3},
  'hypoallergenic': True},
 'relationships': {'group': {'data': {'id': 'f56dc4b1-ba1a-4454-8ce2-bd5d41404a0c',
    'type': 'group'}}}}

In [230]:
m_weight = first_out["attributes"]["male_weight"]["max"]
f_weight = first_out["attributes"]["female_weight"]["max"]

In [232]:
# Calculate diff
diff_weight = m_weight - f_weight
diff_weight

0

#### 3) Extrapolate to all Breed_IDs

In [234]:
### Extrapolate to all breed_IDs

In [260]:
max_differential = 0
breed_ID = ""
breed_name = ""

for obj in json_response_breed["data"]:
    
    m_weight = obj["attributes"]["male_weight"]["max"]
    f_weight = obj["attributes"]["female_weight"]["max"]
    diff_weight = m_weight - f_weight

    if diff_weight > max_differential:
        max_differential = diff_weight
        breed_ID = obj["id"]
        breed_name = obj["attributes"]["name"]

In [295]:
print('Name: '+ breed_name +'; Differential:', max_differential)

Name: Akita; Differential: 10


In [ ]:
### Find top 5 max weight differentials

In [329]:
max_differential_dict = [] 

breed_ID = ""
breed_name = ""

for obj in json_response_breed["data"]:
    
    m_weight = obj["attributes"]["male_weight"]["max"]
    f_weight = obj["attributes"]["female_weight"]["max"]
    diff_weight = m_weight - f_weight
    breed_ID = obj["id"]

    #Append a dictionary in the list (so creates a list of dictionaries)
    max_differential_dict.append({"breed_ID":breed_ID,
                                  "diff_weight":diff_weight})

In [331]:
max_differential_dict

[{'breed_ID': '036feed0-da8a-42c9-ab9a-57449b530b13', 'diff_weight': 0},
 {'breed_ID': 'dd9362cc-52e0-462d-b856-fccdcf24b140', 'diff_weight': 2},
 {'breed_ID': '1460844f-841c-4de8-b788-271aa4d63224', 'diff_weight': 3},
 {'breed_ID': 'e7e99424-d514-4b56-9f0c-05736f6dd22d', 'diff_weight': 10},
 {'breed_ID': '667c7359-a739-4f2b-abb4-98867671e375', 'diff_weight': 0},
 {'breed_ID': '5328d59b-b4e4-48e9-98ec-0545c66c4385', 'diff_weight': 5},
 {'breed_ID': 'f72528b5-a5d7-4a17-b709-aba2db722307', 'diff_weight': 5},
 {'breed_ID': '4524645f-dda7-4031-9272-dee29f5f91ea', 'diff_weight': 0},
 {'breed_ID': 'e1c0664d-aa61-4c85-970d-6c86ba197bee', 'diff_weight': 0},
 {'breed_ID': '8355b9c9-3724-477d-858a-c1c1c0f1743f', 'diff_weight': 0}]

In [355]:
# Sort by 'diff_weight' in descending order
sorted_max_differential_dict = sorted(max_differential_dict, key=lambda x: x['diff_weight'], reverse=True) #lambda is a nonname function
sorted_max_differential_dict[:5]

[{'breed_ID': 'e7e99424-d514-4b56-9f0c-05736f6dd22d', 'diff_weight': 10},
 {'breed_ID': '5328d59b-b4e4-48e9-98ec-0545c66c4385', 'diff_weight': 5},
 {'breed_ID': 'f72528b5-a5d7-4a17-b709-aba2db722307', 'diff_weight': 5},
 {'breed_ID': '1460844f-841c-4de8-b788-271aa4d63224', 'diff_weight': 3},
 {'breed_ID': 'dd9362cc-52e0-462d-b856-fccdcf24b140', 'diff_weight': 2}]

In [ ]:
#In: Single element of list (dict in this case)
#Out: Num to sort the list (the order of the list) 

In [341]:
def listElement_to_singleNum(dict):
    
    diff_weight = dict["diff_weight"]

    return diff_weight

In [349]:
sorted_max_differential_dict = sorted(max_differential_dict,key=listElement_to_singleNum,reverse=True)
sorted_max_differential_dict[:5]

[{'breed_ID': 'e7e99424-d514-4b56-9f0c-05736f6dd22d', 'diff_weight': 10},
 {'breed_ID': '5328d59b-b4e4-48e9-98ec-0545c66c4385', 'diff_weight': 5},
 {'breed_ID': 'f72528b5-a5d7-4a17-b709-aba2db722307', 'diff_weight': 5},
 {'breed_ID': '1460844f-841c-4de8-b788-271aa4d63224', 'diff_weight': 3},
 {'breed_ID': 'dd9362cc-52e0-462d-b856-fccdcf24b140', 'diff_weight': 2}]

## Public API Example #5
#### Paginated APIs 
Common to not return all of the data at once so need to pagninate through the API (can see if paginated if there are links at the bottom)

Q: Paginate through the entire breeds API and return the name of every breed that starts with A, B, or C

In [384]:
pg_num = 2

In [386]:
#Get JSON Response /breeds?page[number]=2 --> getting the next page
headers = {
    'accept': 'application/json' #what type of response I accept
}

params = { #query parameters [AFTER the Question Mark!] 
    'page[number]': pg_num 
}

response = requests.get( #get requests dont have bodies
    'https://dogapi.dog/api/v2/breeds',
    params = params,
    headers = headers
)

json_response_breed = response.json()

In [397]:
pagninated_API_responses = [] #list of each page's API responses

page_numbers = [] #list of all page numbers need to paginate through
page_numbers = list(range(1,5))

[1, 2, 3, 4]

#### 1) Put all Paginated JSON responses in a list

In [399]:
for pg_num in page_numbers:

    #Get JSON Response /breeds?page[number]=2 --> getting the next page
    headers = {
        'accept': 'application/json' #what type of response I accept
    }

    params = { #query parameters [AFTER the Question Mark!] 
        'page[number]': pg_num 
    }

    response = requests.get( #get requests dont have bodies
        'https://dogapi.dog/api/v2/breeds',
        params = params,
        headers = headers
    )

    json_response_breed = response.json()

    pagninated_API_responses.append(json_response_breed)

In [489]:
#Do for one example

In [427]:
pagninated_API_responses[0]["data"][0]

{'id': '036feed0-da8a-42c9-ab9a-57449b530b13',
 'type': 'breed',
 'attributes': {'name': 'Affenpinscher',
  'description': 'The Affenpinscher is a small and playful breed of dog that was originally bred in Germany for hunting small game. They are intelligent, energetic, and affectionate, and make excellent companion dogs.',
  'life': {'max': 16, 'min': 14},
  'male_weight': {'max': 5, 'min': 3},
  'female_weight': {'max': 5, 'min': 3},
  'hypoallergenic': True},
 'relationships': {'group': {'data': {'id': 'f56dc4b1-ba1a-4454-8ce2-bd5d41404a0c',
    'type': 'group'}}}}

In [415]:
pagninated_API_responses[0]["data"][0]["attributes"]["name"]

'Affenpinscher'

#### 2) Define Function to return list of matched names from one JSON paginated response

In [469]:
def names_in_one_page(one_page_json_response):

    list_of_matched_names = []

    for obj in one_page_json_response["data"]:

        #Get name of specific breed
        name = obj["attributes"]["name"]

        #Check if name is STARTS WITH A,B,C (if so, append to list)
        if name.startswith("A") or name.startswith("B") or name.startswith("C"):
            list_of_matched_names.append(name)

    return list_of_matched_names

In [485]:
one_page_example = pagninated_API_responses[3]

In [487]:
names_in_one_page(one_page_example)

['Beauceron',
 'Bedlington Terrier',
 'Belgian Laekenois',
 'Belgian Malinois',
 'Belgian Sheepdog',
 'Belgian Tervuren',
 'Bergamasco Sheepdog',
 'Berger Picard',
 'Bernese Mountain Dog',
 'Bichon Frise']

#### 3) Loop Over all Paginated API Responses

In [509]:
all_matched_names = []

for one_response in pagninated_API_responses:

    #Get associated matched names in that page response
    names = names_in_one_page(one_response)

    all_matched_names.append(names)

In [511]:
all_matched_names

[['Affenpinscher',
  'Afghan Hound',
  'Airedale Terrier',
  'Akita',
  'Alaskan Klee Kai',
  'Alaskan Malamute',
  'American Bulldog',
  'American English Coonhound',
  'American Eskimo Dog',
  'American Foxhound'],
 ['American Hairless Terrier',
  'American Leopard Hound',
  'American Staffordshire Terrier',
  'American Water Spaniel',
  'Anatolian Shepherd Dog',
  'Appenzeller Sennenhund',
  'Australian Cattle Dog',
  'Australian Kelpie',
  'Australian Shepherd',
  'Australian Stumpy Tail Cattle Dog'],
 ['Australian Terrier',
  'Azawakh',
  'Barbado da Terceira',
  'Barbet',
  'Basenji',
  'Basset Fauve de Bretagne',
  'Basset Hound',
  'Bavarian Mountain Scent Houn',
  'Beagle',
  'Bearded Collie'],
 ['Beauceron',
  'Bedlington Terrier',
  'Belgian Laekenois',
  'Belgian Malinois',
  'Belgian Sheepdog',
  'Belgian Tervuren',
  'Bergamasco Sheepdog',
  'Berger Picard',
  'Bernese Mountain Dog',
  'Bichon Frise']]

#### 4) Flatten the List

In [503]:
#Flatten list
def flatten(list_of_lists):

    one_list = []

    for list in list_of_lists:
        for element in list:
            one_list.append(element) 

    return one_list

In [513]:
one_matched_name_list = flatten(all_matched_names)
one_matched_name_list

['Affenpinscher',
 'Afghan Hound',
 'Airedale Terrier',
 'Akita',
 'Alaskan Klee Kai',
 'Alaskan Malamute',
 'American Bulldog',
 'American English Coonhound',
 'American Eskimo Dog',
 'American Foxhound',
 'American Hairless Terrier',
 'American Leopard Hound',
 'American Staffordshire Terrier',
 'American Water Spaniel',
 'Anatolian Shepherd Dog',
 'Appenzeller Sennenhund',
 'Australian Cattle Dog',
 'Australian Kelpie',
 'Australian Shepherd',
 'Australian Stumpy Tail Cattle Dog',
 'Australian Terrier',
 'Azawakh',
 'Barbado da Terceira',
 'Barbet',
 'Basenji',
 'Basset Fauve de Bretagne',
 'Basset Hound',
 'Bavarian Mountain Scent Houn',
 'Beagle',
 'Bearded Collie',
 'Beauceron',
 'Bedlington Terrier',
 'Belgian Laekenois',
 'Belgian Malinois',
 'Belgian Sheepdog',
 'Belgian Tervuren',
 'Bergamasco Sheepdog',
 'Berger Picard',
 'Bernese Mountain Dog',
 'Bichon Frise']

## 6) About API Requests

##### You can also retrieve the header values not just the body

In [370]:
json_response_headers = response.headers

json_response_headers

{'Cache-Control': 'max-age=0, private, must-revalidate', 'Content-Type': 'application/vnd.api+json; charset=utf-8', 'Etag': 'W/"89cf8ea839da513fb87de5c30d99d688"', 'Referrer-Policy': 'strict-origin-when-cross-origin', 'Vary': 'Accept, Origin', 'X-Content-Type-Options': 'nosniff', 'X-Download-Options': 'noopen', 'X-Frame-Options': 'SAMEORIGIN', 'X-Permitted-Cross-Domain-Policies': 'none', 'X-Request-Id': '0fb18da6-a309-4bc9-8ef4-c4e5dfca305e', 'X-Runtime': '0.024973', 'X-Xss-Protection': '0', 'Date': 'Wed, 30 Jul 2025 01:55:52 GMT', 'Transfer-Encoding': 'chunked'}

In [379]:
headers = { #usually includes things like API version #, API KEY
    'X-API-KEY': 'xxxxxxxxxxxxx-xxxxxxxxxxxxxxx-xxxxxxxxxxxx'
}

#params = { #query parameters [AFTER the Question Mark!] 
#    'page[number]': 2 
#}

payload = { #this is the body of the request
    "config": {
        "ip": {
          "include": "flags,history,id",
          "version": "v1"
        },
        "email": {
          "version": "v3"
        },
        "phone": {
          "version": "v2"
        },
        "ip_api": True,
        "email_api": True,
        "phone_api": True,
        "device_fingerprinting": True
      },
    "ip": "11.22.33.44",
    "action_type": "application",
    "user_fullname": "John M Smith",
    "user_firstname": "John",
    "user_middlename": "M",
    "user_lastname": "Smith",
    "user_name": "John Smith",
    "user_dob": "1990-01-01",
    "email": "john.smith@example.com",
    "phone_number": "+1234567890",
    "user_street": "123 Main St",
    "user_street2": "Apt 4B",
    "user_zip": "10001",
    "user_city": "New York",
    "user_region": "NY",
    "user_country": "US",
    "billing_street": "123 Main St",
    "billing_street2": "Apt 4B",
    "billing_zip": "10001",
    "billing_city": "New York",
    "billing_region": "NY",
    "billing_country": "US",
    "billing_phone": "+1234567890",
    "custom_fields": {
        "ssn": "123-45-6789"
  }
}

response = requests.post(
    'https://api.seon.io/SeonRestService/fraud-api/v2',
    #params=params,
    headers=headers,
    data=json.dumps(payload) #body
)

print(response.json()) #will return error as no API key

{'success': False, 'error': {'code': '2002', 'message': 'invalid license key'}, 'data': {}}


## DataFrames Review

#### 1) Reading Data into CSV/XLSX

In [620]:
data = {
    "name": ["Alice", "Bob", "Charlie", "Diana", "Evan"],
    "age": [25, 32, 37, 29, 41],
    "city": ["NY", "LA", "NY", "SF", "LA"],
    "income": [5000, 6000, 5500, 7000, 6500]
}

df = pd.DataFrame(data)

In [ ]:
df = pd.read_csv("data.csv")
df = pd.read_excel("data.xlsx", sheet_name="Sheet1")

#### 2) Select & Filter 

In [622]:
#Choose specific columns
df2 = df[["name", "income"]]

In [626]:
#Filter by specific condition
df2 = df[(df["age"] > 30) & (df["city"] == "NY")] 
df2

,name,age,city,income
2,Charlie,37,NY,5500


#### 3) Add/Rename Columns

In [628]:
#Add new column
df["annual_income"] = df["income"] * 12
df

,name,age,city,income,annual_income
0,Alice,25,NY,5000,60000
1,Bob,32,LA,6000,72000
2,Charlie,37,NY,5500,66000
3,Diana,29,SF,7000,84000
4,Evan,41,LA,6500,78000


In [630]:
#Rename Columns
df = df.rename(columns={'name': 'first_name', 'income': 'mo_income'})
df

,first_name,age,city,mo_income,annual_income
0,Alice,25,NY,5000,60000
1,Bob,32,LA,6000,72000
2,Charlie,37,NY,5500,66000
3,Diana,29,SF,7000,84000
4,Evan,41,LA,6500,78000


#### 4) Apply Functions

In [634]:
#Column wise
df["name_length"] = df["first_name"].apply(len)
df

,first_name,age,city,mo_income,annual_income,name_length
0,Alice,25,NY,5000,60000,5
1,Bob,32,LA,6000,72000,3
2,Charlie,37,NY,5500,66000,7
3,Diana,29,SF,7000,84000,5
4,Evan,41,LA,6500,78000,4


In [644]:
#Row wise
#df["age_plus_income"] = df.apply(lambda row: row["age"] + row["mo_income"], axis=1)
df["age_plus_income"] = df["age"] + df["mo_income"]
df

,first_name,age,city,mo_income,annual_income,name_length,age_plus_income
0,Alice,25,NY,5000,60000,5,5025
1,Bob,32,LA,6000,72000,3,6032
2,Charlie,37,NY,5500,66000,7,5537
3,Diana,29,SF,7000,84000,5,7029
4,Evan,41,LA,6500,78000,4,6541


#### 5) Group and Aggregate

In [652]:
#GROUP BY
df.groupby("city")["mo_income"].mean()

city
LA    6250.0
NY    5250.0
SF    7000.0
Name: mo_income, dtype: float64

In [660]:
#Aggs
result = df["mo_income"].aggregate(['sum', 'mean'])
result

sum     30000.0
mean     6000.0
Name: mo_income, dtype: float64

In [662]:

results2 = df.groupby("city").agg({
                "mo_income": ["mean", "max"],
                "age": "count"
            }).reset_index()
results2

city mo_income         age
            mean   max count
0   LA    6250.0  6500     2
1   NY    5250.0  5500     2
2   SF    7000.0  7000     1

#### 6) Sort

In [648]:
df = df.sort_values(by="mo_income", ascending=False)
df

,first_name,age,city,mo_income,annual_income,name_length,age_plus_income
3,Diana,29,SF,7000,84000,5,7029
4,Evan,41,LA,6500,78000,4,6541
1,Bob,32,LA,6000,72000,3,6032
2,Charlie,37,NY,5500,66000,7,5537
0,Alice,25,NY,5000,60000,5,5025


#### 7) Join DFs

In [666]:
city_data = pd.DataFrame({
    "city": ["NY", "LA", "SF"],
    "region": ["East", "West", "West"]
})
city_data

,city,region
0,NY,East
1,LA,West
2,SF,West


In [668]:
merged = pd.merge(df, city_data, on="city", how="left")
merged

,first_name,age,city,mo_income,annual_income,name_length,age_plus_income,region
0,Diana,29,SF,7000,84000,5,7029,West
1,Evan,41,LA,6500,78000,4,6541,West
2,Bob,32,LA,6000,72000,3,6032,West
3,Charlie,37,NY,5500,66000,7,5537,East
4,Alice,25,NY,5000,60000,5,5025,East


#### 8) Handle Missing Data

In [ ]:
df.dropna()   # Drop rows with NaNs
df.fillna(0)  # Replace NaNs with 0

## 7) DF Example

#### Q: Find all dates ID with higher temperature compared to its previous dates (yesterday)

In [599]:
weather = pd.DataFrame({
    "id": [1, 2, 3, 4],
    "recordDate": ["2015-01-01", "2015-01-02", "2015-01-03", "2015-01-04"],
    "temperature": [10, 25, 20, 30]
})

weather

,id,recordDate,temperature
0,1,2015-01-01,10
1,2,2015-01-02,25
2,3,2015-01-03,20
3,4,2015-01-04,30


In [601]:
# Ensure the 'recordDate' column is a datetime type
weather['recordDate'] = pd.to_datetime(weather['recordDate'])
# Rename column
weather = weather.rename(columns={'temperature': 'today_temp'})

weather

,id,recordDate,today_temp
0,1,2015-01-01,10
1,2,2015-01-02,25
2,3,2015-01-03,20
3,4,2015-01-04,30


In [583]:
# Create a copy of the weather DataFrame with a 1 day shift 
weather_shifted = weather.copy()
weather_shifted['recordDate'] = weather_shifted['recordDate'] + pd.to_timedelta(1, unit='D')
weather_shifted

,id,recordDate,temperature
0,1,2015-01-02,10
1,2,2015-01-03,25
2,3,2015-01-04,20
3,4,2015-01-05,30


In [587]:
merged_df = pd.merge(weather, weather_shifted, on='recordDate')
merged_df

,id_x,recordDate,temperature_x,id_y,temperature_y
0,2,2015-01-02,25,1,10
1,3,2015-01-03,20,2,25
2,4,2015-01-04,30,3,20


In [585]:
# Merging the DataFrames on the 'recordDate' column to find consecutive dates
merged_df = pd.merge(weather, weather_shifted, on='recordDate', suffixes=('_today', '_yesterday'))
merged_df

,id_today,recordDate,temperature_today,id_yesterday,temperature_yesterday
0,2,2015-01-02,25,1,10
1,3,2015-01-03,20,2,25
2,4,2015-01-04,30,3,20


In [581]:
# Finding rows where the temperature is greater on the current day compared to the previous day
result = merged_df[merged_df['temperature_today'] > merged_df['temperature_yesterday']][['id_today']].rename(columns={'id_today': 'Id'})
result

,Id
0,2
2,4
